In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import fastai
import fastai.basics as fai
import fastai.collab as fc
import fastai.tabular as ft

from pathlib import Path
from shutil import copyfile
import random
import gc

import torch
import torch.nn.functional as F
import torch.nn as nn

import torchvision
import torchvision.utils as utils
import torchvision.models
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [3]:
path = Path('AnimeRatings/')

In [4]:
ratings = pd.read_csv(path/"rating.csv")

Quitamos los -1's, porque, al menos por el momento, no nos sirven de nada (esto se podría investigar. Quizás se podría cambiar el -1 por algo como el promedio de ese anime, o algo entre el promedio de ese anime y el promedio global, porque si no calificas algo que sí viste es porque quizás no te impresionó suficiente)

In [5]:
ratings = ratings[ratings['rating'] != -1]

In [6]:
ratings.head()

,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
153,2,11771,10


In [7]:
animes = pd.read_csv("AnimeRatings/anime.csv")

In [8]:
animes.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [9]:
ratings = ratings.merge(animes[["anime_id", "name"]])

In [10]:
ratings.head()

,user_id,anime_id,rating,name
0,1,8074,10,Highschool of the Dead
1,3,8074,6,Highschool of the Dead
2,5,8074,2,Highschool of the Dead
3,12,8074,6,Highschool of the Dead
4,14,8074,6,Highschool of the Dead


In [86]:
data = fc.CollabDataBunch.from_df(ratings, seed=666, valid_pct=0.1, item_name="name", bs=512)

In [12]:
data.show_batch()

user_id,name,target
49990,Howl no Ugoku Shiro,6.0
72557,Air,6.0
21307,Strait Jacket,8.0
5816,Mobile Suit Gundam Wing,8.0
34588,Karigurashi no Arrietty,5.0


In [22]:
fc.collab_learner??

Signature:
fc.collab_learner(
    data,
    n_factors:int=None,
    use_nn:bool=False,
    emb_szs:Dict[str, int]=None,
    layers:Collection[int]=None,
    ps:Collection[float]=None,
    emb_drop:float=0.0,
    y_range:Union[Tuple[float, float], NoneType]=None,
    use_bn:bool=True,
    bn_final:bool=False,
    **learn_kwargs,
) -> fastai.basic_train.Learner
Source:   
def collab_learner(data, n_factors:int=None, use_nn:bool=False, emb_szs:Dict[str,int]=None, layers:Collection[int]=None, 
                   ps:Collection[float]=None, emb_drop:float=0., y_range:OptRange=None, use_bn:bool=True, 
                   bn_final:bool=False, **learn_kwargs)->Learner:
    "Create a Learner for collaborative filtering on `data`."
    emb_szs = data.get_emb_szs(ifnone(emb_szs, {}))
    u,m = data.train_ds.x.classes.values()
    if use_nn: model = EmbeddingNN(emb_szs=emb_szs, layers=layers, ps=ps, emb_drop=emb_drop, y_range=y_range, 
                                   use_bn=use_bn, bn_final=bn_fi

In [23]:
fc.EmbeddingDotBias??

Init signature:
fc.EmbeddingDotBias(
    n_factors:int,
    n_users:int,
    n_items:int,
    y_range:Tuple[float, float]=None,
)
Source:        
class EmbeddingDotBias(nn.Module):
    "Base dot model for collaborative filtering."
    def __init__(self, n_factors:int, n_users:int, n_items:int, y_range:Tuple[float,float]=None):
        super().__init__()
        self.y_range = y_range
        (self.u_weight, self.i_weight, self.u_bias, self.i_bias) = [embedding(*o) for o in [
            (n_users, n_factors), (n_items, n_factors), (n_users,1), (n_items,1)
        ]]

    def forward(self, users:LongTensor, items:LongTensor) -> Tensor:
        dot = self.u_weight(users)* self.i_weight(items)
        res = dot.sum(1) + self.u_bias(users).squeeze() + self.i_bias(items).squeeze()
        if self.y_range is None: return res
        return torch.sigmoid(res) * (self.y_range[1]-self.y_range[0]) + self.y_range[0]
File:           ~/anaconda3/envs/fastai/lib/python3.6/site-packages/fastai/collab.

In [80]:
class CollabFilterModel(nn.Module):
    def __init__(self, num_users, num_items, embedding_size=10):
        super().__init__()
        self.user_weights = nn.Embedding(num_users, embedding_size)
        self.item_weights = nn.Embedding(num_items, embedding_size)
        
    def forward(self,users:fai.LongTensor, items:fai.LongTensor):
        #print("users shape = ", self.user_weights(users).shape)
        #print("items shape = ", self.item_weights(items).shape)
        
        return (self.user_weights(users)*self.item_weights(items)).sum(dim=1)

In [37]:
users,items = data.train_ds.x.classes.values()

In [38]:
items

array(['#na#', '&quot;0&quot;', '&quot;Aesop&quot; no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu',
       '&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi', ..., 'xxxHOLiC Movie: Manatsu no Yoru no Yume',
       'xxxHOLiC Rou', 'xxxHOLiC Shunmuki', '◯'], dtype=object)

In [81]:
model = CollabFilterModel(len(users), len(items), embedding_size=10).cuda()

In [46]:
batch = next(iter(data.train_dl))

In [47]:
(U,A),R = batch

In [76]:
model(U,A).shape

users shape =  torch.Size([64, 10])
items shape =  torch.Size([64, 10])


torch.Size([64])

In [66]:
R.shape

torch.Size([64])

In [87]:
gc.collect()

8

In [89]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [90]:
smooth_loss = 50
with tqdm(data.train_dl) as pbar:
    for batch in pbar:
        optimizer.zero_grad()
        (U,A),R = batch
        loss = F.mse_loss(model(U,A),R)
        loss.backward()
        optimizer.step()
        smooth_loss = loss.item()*0.1 + smooth_loss*0.9
        pbar.set_postfix(loss=smooth_loss)
    